### This notebook combines three separate scripts written for the live population dashboard project. The first part contains the code for some function defined and used later in the project. It also contains the websites from which data was scraped. The second part contains the main scripts that scrapes the data, cleans the data and writes it into postgress database. The third part contains the script to grab OSM data via Geofabrik and write into a postgresql database.

**Contributor: Selorm Komla Darkey*

**Part One:**

In [2]:
# Importing library
import psycopg2


# Defining Connect to the database
def get_connection():
    conn = psycopg2.connect(
        host="localhost",
        database="geodb",
        user="postgres",
        password="mydbms1"
        )
    return conn


schema_containing_tables = "hft"


# DATA BANK FOR ALL STATISTIK WEBSITE LINKS AND TABLE NAMES #
# ----------- POPULATION BY NATIONALITY LINKS  ------------------#
nat_links = [
        "https://www.statistik-bw.de/BevoelkGebiet/MigrNation/01035010.tab?R=RB1",   # Stuttgart Administrative
        "https://www.statistik-bw.de/BevoelkGebiet/MigrNation/01035010.tab?R=RB2",   # Karlsruhe Administrative
        "https://www.statistik-bw.de/BevoelkGebiet/MigrNation/01035010.tab?R=KR116",  # City District Esslingen
        "https://www.statistik-bw.de/BevoelkGebiet/MigrNation/01035010.tab?R=KR111",  # City District of Stuttgart
        "https://www.statistik-bw.de/BevoelkGebiet/MigrNation/01035010.tab?R=KR212"    # City District of Karlsruhe
 ]
# ---------- POPULATIONS BY AGE GROUPS LINKS  --------------------#
age_group_links = [
        "https://www.statistik-bw.de/BevoelkGebiet/Alter/01035410.tab?R=RB1",       # Stuttgart Administrative
        "https://www.statistik-bw.de/BevoelkGebiet/Alter/01035410.tab?R=RB2",       # Karlsruhe Administrative
        "https://www.statistik-bw.de/BevoelkGebiet/Alter/01035410.tab?R=KR116",     # City District Esslingen
        "https://www.statistik-bw.de/BevoelkGebiet/Alter/01035410.tab?R=KR111",     # City District of Stuttgart
        "https://www.statistik-bw.de/BevoelkGebiet/Alter/01035410.tab?R=KR212"      # City District of Karlsruhe
 ]
# ---------- TABLE NAMES TO BE USED FOR THE CSV  --------------------#
table_names = [["popbynat_stuttgartadmin", "popbyagegroups_stuttgartadmin"],
               ["popbynat_karlsruheadmin", "popbyagegroups_karlsruheadmin"],
               ["popbynat_esslingencity", "popbyagegroups_esslingencity"],
               ["popbynat_stuttgartcity", "popbyagegroups_stuttgartcity"],
               ["popbynat_karlsruhecity", "popbyagegroups_karlsruhecity"]]

lat_lon = [[48.76998, 9.1752525],       # stuttgart
           [49.006889, 8.403653],       # karlsruhe
           [48.73961, 9.30473],         # esslingen
           [48.76998, 9.1752525],       # stuttgart_city
           [49.006889, 8.403653]]       # karlsruhe_city

special_id = [21, 15, 22, 29, 738]  # Stuttgart_admin, karlsruhe, esslingen, stuttgart_city,


def create_table_age_groups(schema, table_name):

    conn = get_connection()
    cur = conn.cursor()

    create_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {schema}.{table_name}
(
        id integer NOT NULL GENERATED ALWAYS AS IDENTITY ( INCREMENT 1 START 1 MINVALUE 1 MAXVALUE 2147483647 CACHE 1 ),
        age_years integer,      
        total_pop integer,
        under15 integer,
        fifteenToEighteen integer,
        EighteenToTwentyfive integer,
        twentyfiveToforty integer,
        fortyTosixtyfive integer,
        sixtyfiveAndAbove integer,
        latitude double precision,
        longitude double precision,
        index integer,
        createdon date DEFAULT CURRENT_DATE,
        updatedon date DEFAULT CURRENT_DATE,
        CONSTRAINT "{table_name}_pkey" PRIMARY KEY (id)
)

        TABLESPACE pg_default;
        """
    cur.execute(create_table_sql)
    conn.commit()
    cur.close()
    conn.close()


def create_table_nats(schema, table_name):

    conn = get_connection()
    cur = conn.cursor()

    create_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {schema}.{table_name}
(
        id integer NOT NULL GENERATED ALWAYS AS IDENTITY ( INCREMENT 1 START 1 MINVALUE 1 MAXVALUE 2147483647 CACHE 1 ),
        nat_years integer,
        total_pop integer,
        total_masculine integer,
        total_german integer,
        german_masculine integer,
        total_foreigners integer,
        foreigner_masculine integer,
        latitude double precision,
        longitude double precision,
        index integer,
        createdon date DEFAULT CURRENT_DATE,
        updatedon date DEFAULT CURRENT_DATE,
        CONSTRAINT "{table_name}_pkey" PRIMARY KEY (id)
)

        TABLESPACE pg_default;
        """
    cur.execute(create_table_sql)
    conn.commit()
    cur.close()
    conn.close()


# insects data into population tables. takes a list, schema name, table name
def insert_data_a(data_to_insert, schema, table_name):

    conn = get_connection()
    cur = conn.cursor()

    insert_sql = f"""
                    INSERT INTO {schema}."{table_name}"(
                    age_years, total_pop, under15, fifteenToEighteen, EighteenToTwentyfive, twentyfiveToforty,
                    fortyTosixtyfive, sixtyfiveAndAbove, latitude, longitude, index)
                    VALUES ({data_to_insert[0]}, {data_to_insert[1]}, {data_to_insert[2]}, {data_to_insert[3]},
                            {data_to_insert[4]}, {data_to_insert[5]}, {data_to_insert[6]}, {data_to_insert[7]},
                            {data_to_insert[8]}, {data_to_insert[9]}, {data_to_insert[10]});
                """

    cur.execute(insert_sql)
    conn.commit()
    cur.close()
    conn.close()


# SECOND INSECT DATA #
def insert_data_n(data_to_insert, schema, table_name):     # for population by nationality

    conn = get_connection()
    cur = conn.cursor()

    insert_sql = f"""
                    INSERT INTO {schema}.{table_name}(
                    nat_years, total_pop, total_masculine, total_german, german_masculine, total_foreigners,
                                    foreigner_masculine, latitude, longitude, index)
                    VALUES ({data_to_insert[0]}, {data_to_insert[1]}, {data_to_insert[2]}, {data_to_insert[3]},
                            {data_to_insert[4]}, {data_to_insert[5]}, {data_to_insert[6]},
                            {data_to_insert[7]}, {data_to_insert[8]}, {data_to_insert[9]});
                """

    cur.execute(insert_sql)
    conn.commit()
    cur.close()
    conn.close()


def truncate_table(schema, table_name):

    conn = get_connection()
    cur = conn.cursor()

    truncate_sql = f"""
                    truncate table {schema}.{table_name}

                """
    cur.execute(truncate_sql)
    conn.commit()
    cur.close()
    conn.close()


ModuleNotFoundError: No module named 'psycopg2'

**Part Two:**

In [ ]:
# importing relevant libraries
from bs4 import BeautifulSoup
import requests
import Actions
schema = Actions.schema_containing_tables                # declare database schema here


for i in range(len(Actions.nat_links)):

    nat_table_link = Actions.nat_links[i]
    pop_table_link = Actions.age_group_links[i]
    nat_table_name = Actions.table_names[i][0]            # nationality table names
    ageGr_table_name = Actions.table_names[i][1]          # ageGroups table names

    # making request to the website
    response = requests.get(nat_table_link)                # nationality
    response2 = requests.get(pop_table_link)               # age Groups

    # getting the web content as text
    pop_by_nat_html = response.text             # pop by nationality html from BW website
    pop_by_ageGroups = response2.text           # pop by age groups html from BW website

    # Using beautiful soup and html parser to pass the response as html
    soup = BeautifulSoup(pop_by_nat_html, "html.parser")            # For population by nationality
    soup2 = BeautifulSoup(pop_by_ageGroups, "html.parser")          # For population by age groups

    # grabbing the needed table rows from the website contents
    pop_by_nat_table_rows = soup.find_all(name="tr")[25:]          # indexing to print only from the desired year
    pop_by_ageG_table_rows = soup2.find_all(name="tr")[6:]

    # create table if it does not exist
    Actions.create_table_age_groups(schema=schema, table_name=ageGr_table_name)

    # truncating old data in table to be replaced with new data
    Actions.truncate_table(schema=schema, table_name=ageGr_table_name)    # deletes existing records in table

    # Write the data rows to the database
    year_omit = '1987'  # removing some duplicate years in the data
    counts = 0
    for row in pop_by_ageG_table_rows:
        age_list = []
        print("=========")

        age_year = row.find(name="th").getText()[:4].replace("*", "")
        if age_year == year_omit and counts == 0:
            counts += 1
            continue
        total_pop = row.find_all(name="td")[0].getText().replace(".", "")
        under15 = row.find_all(name="td")[1].getText().replace(".", "")
        fifteenToEighteen = row.find_all(name="td")[2].getText().replace(".", "")
        EighteenToTwentyfive = row.find_all(name="td")[3].getText().replace(".", "")
        twentyfiveToforty = row.find_all(name="td")[4].getText().replace(".", "")
        fortyTosixtyfive = row.find_all(name="td")[5].getText().replace(".", "")
        sixtyfiveAndAbove = row.find_all(name="td")[6].getText().replace(".", "")
        latitude = Actions.lat_lon[i][0]
        longitude = Actions.lat_lon[i][1]
        index = Actions.special_id[i]

        age_list.extend(
            [age_year, total_pop, under15, fifteenToEighteen, EighteenToTwentyfive, twentyfiveToforty, fortyTosixtyfive,
             sixtyfiveAndAbove, latitude, longitude, index])  # latitude, longitude, index
        # print(age_list)

        Actions.insert_data_a(data_to_insert=age_list, schema=schema, table_name=ageGr_table_name)


# next table
    # create pop by nat table if it does not exist
    Actions.create_table_nats(schema=schema, table_name=nat_table_name)

    # truncate existing table
    Actions.truncate_table(schema=schema, table_name=nat_table_name)     # deletes existing record in table

    # Write the data rows to the database
    char = "*"
    for rows in pop_by_nat_table_rows:
        nat_list = []
        print("=========")
        nat_year = rows.find(name="th").getText()
        if char in nat_year:
            continue
        total_pop = rows.find_all(name="td")[0].getText().replace(".", "")
        total_masculine = rows.find_all(name="td")[1].getText().replace(".", "")
        total_german = rows.find_all(name="td")[2].getText().replace(".", "")
        german_masculine = rows.find_all(name="td")[3].getText().replace(".", "")
        total_foreigners = rows.find_all(name="td")[4].getText().replace(".", "")
        foreigner_masculine = rows.find_all(name="td")[5].getText().replace(".", "")
        latitude = Actions.lat_lon[i][0]
        longitude = Actions.lat_lon[i][1]
        index = Actions.special_id[i]

        nat_list.extend(
            [nat_year, total_pop, total_masculine, total_german, german_masculine, total_foreigners,
             foreigner_masculine, latitude, longitude, index])

        Actions.insert_data_n(data_to_insert=nat_list, schema=schema, table_name=nat_table_name)
        

**Part Three:**

In [ ]:
# This script get the osm data as zip file, unzips it into a folder and writes it to a database.
# it makes use of the request, zipfile, geopandas and sqlalchemy libraries.

from sqlalchemy import create_engine
import requests
import zipfile
import geopandas as gpd
import os


# Parameters
osm_site = "https://download.geofabrik.de/europe/germany/baden-wuerttemberg/stuttgart-regbez-latest-free.shp.zip"
user = "postgres"
password = "XXXXX"
host = "localhost"
port = 5432
database = "geodb"


# using the sqlalchemy create_engine method to create a connection
conn = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(conn)


# Download the OSM data file
response = requests.get(osm_site)
file_path = "osm_data.zip"

if response.status_code == 200:
    with open(file_path, "wb") as file:
        file.write(response.content)

# Extract the OSM zip data to a folder
extracted_folder = "extracted_data"

with zipfile.ZipFile(file_path, "r") as zip_ref:
    zip_ref.extractall(extracted_folder)


# defining a function to print all files in a folder. Here I used it to get all files in extracted data folder
def print_file_names(folder):
    # create empty list to store file names
    all_files_names = []
    # check first if folder is valid
    if not os.path.isdir(folder):
        print(f"Error: '{folder}' is not a valid directory.")
        return

    # Getting the list of files in the folder
    files_folder = os.listdir(folder)

    # Iterating over the files and printing their names
    for files in files_folder:
        file_path_ = os.path.join(folder, files)
        if os.path.isfile(file_path_):
            all_files_names.append(file)
    return all_files_names


my_folder = r"./extracted_data"
file_names = print_file_names(my_folder)
print(file_names)

count = 0   # added to enable me print only the desire number of files in folder
# getting only the shapefile names among the file.
for i in file_names:
    # if ".shp" in i and count < 2:           # I added the count to get just the first two shapefiles
    if ".shp" in i:
        shape_file_name = i
        # print(shape_file_name)
        table_name = shape_file_name.replace("_free_1.shp", "")  # I trim the filename to be used as table names in
        # the DB

    # Using GeoPandas to read shapefile using
#    gdf = gpd.read_file("./extracted_data/gis_osm_buildings_a_free_1.shp")   # for writing only one
        file_path = os.path.join(my_folder, shape_file_name)
        gdf = gpd.read_file(file_path)

    # Import the shapefile into the postgresSQL database using SQLAlchemy ORM library
        gdf.to_postgis(name=table_name, con=engine, schema="hft")
        print("success")
        # count += 1